In [2]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
import pandas as pd 
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# import sklearn.external.joblib as extjoblib
import joblib

In [4]:
encoded_label_dict = {"CG" : 0, "OR" : 1}
def encode_label(x):
    return encoded_label_dict.get(x,-1)

In [5]:
df = pd.read_csv("fake reviews dataset.csv")

In [6]:
df["target"] = df["label"].apply(lambda x: encode_label(x))

In [7]:
train, test = train_test_split(df, test_size=0.2, shuffle=True, stratify=None, random_state=2021)

In [8]:
train.head()

,category,rating,label,text,target
29115,Books_5,positive,OR,Nora Roberts never disappoints! Loved the book...,1
31611,Books_5,positive,OR,This was my first time reading this classic an...,1
16922,Tools_and_Home_Improvement_5,positive,CG,"Bought this for my dad, who uses the tools in ...",0
5946,Sports_and_Outdoors_5,positive,CG,These are great beanies. I use them as an exer...,0
24761,Kindle_Store_5,positive,CG,Lots of twists and turns that will make you fe...,0


In [9]:
len(train),len(test)

(32345, 8087)

In [10]:
COMMENT = 'text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

C:\Users\guris\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [20]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [21]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [22]:
trn_term_doc, test_term_doc

(<32345x88639 sparse matrix of type '<class 'numpy.float64'>'
 	with 3307646 stored elements in Compressed Sparse Row format>,
 <8087x88639 sparse matrix of type '<class 'numpy.float64'>'
 	with 807555 stored elements in Compressed Sparse Row format>)

In [23]:
# naive bayes feature
def pr(y_i, y):
    p = x[y==y_i].sum(0) # count vectors formula P = Summation( i:y (i))=
    return (p+1) / ((y==y_i).sum()+1)

In [24]:
x = trn_term_doc
test_x = test_term_doc

In [25]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y)) # log count ratio
#     ratio of feature in positive documents is the number of times a positive document has a feature divided by the number of positive documents.
    m = LogisticRegression(C=4,solver ='saga' )
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [26]:
m,r = get_mdl(train["target"])
preds_probas = m.predict_proba(test_x.multiply(r))[:,1]

In [27]:
preds = [1 if prob>=0.5 else 0 for prob in preds_probas]

In [28]:
from sklearn.metrics import confusion_matrix
y_true = test.target.values
y_pred = preds
confusion_matrix(y_true,y_pred)

array([[3743,  267],
       [ 159, 3918]], dtype=int64)

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [30]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 94.73228638555706; Precision:93.62007168458781; Recall:96.10007358351729


In [31]:
print(classification_report(y_true, y_pred, target_names=["CG","OR"]))


              precision    recall  f1-score   support

          CG       0.96      0.93      0.95      4010
          OR       0.94      0.96      0.95      4077

    accuracy                           0.95      8087
   macro avg       0.95      0.95      0.95      8087
weighted avg       0.95      0.95      0.95      8087



In [32]:
with open('model.pkl','wb') as files:
    pickle.dump(m,files)

In [2]:
import pickle
model = pickle.load(open('model.pkl', 'rb'))

C:\Users\guris\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
C:\Users\guris\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
